In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout,GRU, Embedding, Conv1D,Conv2D, Conv3D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop, SGD





In [ ]:
print(tf.version)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
#Cargamos el dataset
import pathlib
data_dir = r"C:\Users\Miguel\OneDrive - Universidad de Oviedo\TFG\Bases de datos\ICHBI\DATASETS\CICLOS_SINFILTRO\Balanceado_2_clases\dataset"
data_dir = pathlib.Path(data_dir)


In [ ]:
#comprobamos que esta cargado correctamente
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)


In [ ]:
#obtenemos un ejemplo
crackles = list(data_dir.glob('normal/*'))
PIL.Image.open(str(crackles[0]))


In [ ]:
altura_imagen, anchura_imagen= 224,224
batch_size=64
train_ds= tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=17,
    label_mode='categorical',
    image_size=(altura_imagen,anchura_imagen),
    batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=17,
    label_mode='categorical',
    image_size=(altura_imagen, anchura_imagen),
    batch_size=batch_size)


In [ ]:
class_names=train_ds.class_names
print(class_names)

In [ ]:

resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3),pooling='avg', classes=2,weights='imagenet')

for layer in pretrained_model.layers:
    layer.trainable=False

resnet_model.add(pretrained_model)

#############Base################
#resnet_model.add(Flatten())
#resnet_model.add(Dense(1024, activation='relu'))
#################################

#############2D2Drop##############
resnet_model.add(Flatten())
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dropout(0.5))
###################################

##################Progresivo####################
#resnet_model.add(Dense(2048, activation='relu'))
#resnet_model.add(Dropout(0.5))
#resnet_model.add(Dense(1024, activation='relu'))
#resnet_model.add(Dropout(0.25))
#resnet_model.add(Dense(512, activation='relu'))
#resnet_model.add(Dropout(0.1))
################################################


resnet_model.add(Dense(2, activation='softmax'))


In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=100
history= resnet_model.fit(train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
fig1=plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','validation'])
plt.show()

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.axis(ymin=0.1, ymax=3)
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()


In [ ]:
resnet_model.save(
    r'C:\Users\Miguel\OneDrive - Universidad de Oviedo\TFG\Bases de datos\Codigo\ModelosEntrenados\ResNet50_Balanced2_Butter_Prog_80_20.h5')
